In [ ]:
from transformers import SeamlessM4Tv2Model, AutoProcessor
import torch

processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
model = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large")

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [ ]:
# This is a partial list.  A complete list is available on hugging face
# # https://huggingface.co/facebook/seamless-m4t-v2-large
lang_list = [{'sym': 'eng', 'language':'English'},
             {'sym': 'cmn', 'language':'Chinese'},
             {'sym': 'tha', 'language':'Thai'},
             {'sym': 'deu', 'language':'German'},
             {'sym': 'rus', 'language':'Russian'},
             {'sym': 'fra', 'language':'French'},
             {'sym': 'spa', 'language':'Spanish'},
             {'sym': 'jpn', 'language':'Japanese'},
             {'sym': 'vie', 'language':'Vietnamese'}]

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
# https://huggingface.co/facebook/seamless-m4t-v2-large

# Create the dropdown widget
dropdown_sts = widgets.Dropdown(options=[(item['language'], index) for index, item in enumerate(lang_list)],
                            value=0, description='Select a target language for speech to speech:')
dropdown_t_ttt = widgets.Dropdown(options=[(item['language'], index) for index, item in enumerate(lang_list)],
                            value=0, description='Target language:')
dropdown_s_ttt = widgets.Dropdown(options=[(item['language'], index) for index, item in enumerate(lang_list)],
                            value=0, description='Source language:')
dropdown_t_tts = widgets.Dropdown(options=[(item['language'], index) for index, item in enumerate(lang_list)],
                            value=0, description='Target language:')
dropdown_s_tts = widgets.Dropdown(options=[(item['language'], index) for index, item in enumerate(lang_list)],
                            value=0, description='Source language:')
sym_sts = 'eng'
sym_t_ttt = 'eng'
sym_s_ttt = 'eng'
sym_t_tts = 'eng'
sym_s_tts = 'eng'

# Function to handle changes in the dropdown_sts value
def on_dropdown_sts_change(change):
    global sym_sts
    selected_index = change['new']
    selected_option = lang_list[selected_index]
    text = f"Selected option: {selected_option}"
    sym_sts = selected_option['sym']
    language = selected_option['language']
    # text_box11.value = sym_sts + '  ' + language

def on_dropdown_t_ttt_change(change):
    global sym_t_ttt
    selected_index = change['new']
    selected_option = lang_list[selected_index]
    text = f"Selected option: {selected_option}"
    sym_t_ttt = selected_option['sym']
    language = selected_option['language']
    # text_box11.value = sym_t_ttt + '  ' + language

def on_dropdown_s_ttt_change(change):
    global sym_s_ttt
    selected_index = change['new']
    selected_option = lang_list[selected_index]
    text = f"Selected option: {selected_option}"
    sym_s_ttt = selected_option['sym']
    language = selected_option['language']
    # text_box11.value = sym_s_ttt + '  ' + language

def on_dropdown_t_tts_change(change):
    global sym_t_tts
    selected_index = change['new']
    selected_option = lang_list[selected_index]
    text = f"Selected option: {selected_option}"
    sym_t_tts = selected_option['sym']
    language = selected_option['language']
    # text_box11.value = sym_t_tts + '  ' + language

def on_dropdown_s_tts_change(change):
    global sym_s_tts
    selected_index = change['new']
    selected_option = lang_list[selected_index]
    text = f"Selected option: {selected_option}"
    sym_s_tts = selected_option['sym']
    language = selected_option['language']
    # text_box11.value = sym_s_tts + '  ' + language
    

# Attach the function to the dropdown_sts's observe method
dropdown_sts.observe(on_dropdown_sts_change, names='value')
dropdown_t_ttt.observe(on_dropdown_t_ttt_change, names='value')
dropdown_s_ttt.observe(on_dropdown_s_ttt_change, names='value')
dropdown_t_tts.observe(on_dropdown_t_tts_change, names='value')
dropdown_s_tts.observe(on_dropdown_s_tts_change, names='value')

# text_box11 = widgets.Text(value='Initial Text11', description='Text Box11:')
# text_box11.layout.width = '50%'  # Adjust the width as needed

dropdown_sts.layout.width = '500px'  # Adjust the width as needed
# Display the dropdown_sts
# display(dropdown_sts)
# display(widgets.HBox([text_box11]))

# selecte language for three text boxes:  stt then tts
txt1_lang = 'eng'
txt2_lang = 'cmn'
txt3_lang = 'fra'


In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
import sounddevice as sd
import numpy as np
import threading

# Create an HTML widget for the centered app title
centered_app_title = HTML("<h2 style='color:blue; text-align:center;'>SeamlessM4T-large v2 Model Demonstration</h2>")

# Display the centered app title
display(centered_app_title)

# Set the sample rate and other parameters
# sample_rate = 44100  # Adjust as needed
sample_rate = model.config.sampling_rate

# Initial state
recording = False

# Variables to store the recorded audio
audio_data = np.array([])
recording_stream = None

def tts_translate(change):
    # process input
    text_inputs = processor(text = text_box_tts.value, src_lang=sym_s_tts, return_tensors="pt").to(device)
    # text_inputs = processor(text = text_box_tts.value, return_tensors="pt").to(device)
    output_tokens = model.generate(**text_inputs, tgt_lang=sym_t_tts, generate_speech=False)
    translated_text_from_text = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
    play_text_src_tgt(translated_text_from_text, sym_s_tts, sym_t_tts)
    # text_box_tts.value = f"{translated_text_from_text}"

def ttt_translate(change):
    # process input
    text_inputs = processor(text = text_box_ttt.value, src_lang=sym_s_ttt, return_tensors="pt").to(device)
    # text_inputs = processor(text = text_box_ttt.value, return_tensors="pt").to(device)
    output_tokens = model.generate(**text_inputs, tgt_lang=sym_t_ttt, generate_speech=False)
    translated_text_from_text = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
    text_box_ttt.value = f"{translated_text_from_text}"

def toggle_recording(change):
    global recording_stream
    global recording
    recording = not recording
    if recording:
        start_button.description = "Stop Speaking"
        # global audio_data, recording_stream
        global audio_data
        audio_data = np.array([])  # Clear previous recordings
        sd.default.samplerate = sample_rate
        sd.default.channels = 1
        recording_stream = sd.InputStream(callback=callback)
        recording_stream.start()
    else:
        start_button.description = "Start Speaking"
        # global recording_stream
        if recording_stream:
            recording_stream.stop()
            recording_stream.close()
            play_recorded_audio()
    
# Callback function for streaming audio
def callback(indata, frames, time, status):
    global audio_data
    if status:
        print(status)
    audio_data = np.append(audio_data, indata.flatten())

# Function to play  from given text
def play_text_src_tgt(text, src, tgt):
    text_inputs = processor(text = text, src_lang=src, return_tensors="pt").to(device)
    # generate translation audio
    audio_array_from_text = model.generate(**text_inputs, tgt_lang=tgt)[0].cpu().numpy().squeeze()
    # Play the audio using sounddevice
    sd.play(audio_array_from_text, sample_rate)
    sd.wait()  # Wait for playback to finish

# Function to play the recorded audio
def play_recorded_audio():
    sd.play(audio_data, sample_rate)
    sd.wait()
    audio_sample = audio_data
    # audio_sample = audio_array_from_text
    # process input
    # audio_inputs = processor(audios=audio_sample["array"], return_tensors="pt").to(device)
    # audio_inputs = processor(audios=audio_sample, sampling_rate=sample_rate, return_tensors="pt").to(device)
    audio_inputs = processor(audios=audio_sample, sampling_rate=sample_rate, return_tensors="pt").to(device)
    text_box1.value = 'processing ...'
    text_box2.value = 'processing ...'
    text_box3.value = 'processing ...'

    # audio_array_from_audio = model.generate(**audio_inputs, tgt_lang="cmn")[0].cpu().numpy().squeeze()
    audio_array_from_audio = model.generate(**audio_inputs, tgt_lang=sym_sts)[0].cpu().numpy().squeeze()
    sd.play(audio_array_from_audio, sample_rate)
    sd.wait()
    
    # generate translation speech to text
    output_tokens_txt1 = model.generate(**audio_inputs, tgt_lang=txt1_lang, generate_speech=False)
    output_tokens_txt2 = model.generate(**audio_inputs, tgt_lang=txt2_lang, generate_speech=False)
    output_tokens_txt3 = model.generate(**audio_inputs, tgt_lang=txt3_lang, generate_speech=False)
    translated_text_from_audio_txt1 = processor.decode(output_tokens_txt1[0].tolist()[0], skip_special_tokens=True)
    translated_text_from_audio_txt2 = processor.decode(output_tokens_txt2[0].tolist()[0], skip_special_tokens=True)
    translated_text_from_audio_txt3 = processor.decode(output_tokens_txt3[0].tolist()[0], skip_special_tokens=True)

    to_text_txt1 = f"{translated_text_from_audio_txt1}"
    to_text_txt2 = f"{translated_text_from_audio_txt2}"
    to_text_txt3 = f"{translated_text_from_audio_txt3}"
    text_box1.value = to_text_txt1
    text_box2.value = to_text_txt2
    text_box3.value = to_text_txt3
    play_text_src_tgt(to_text_txt1, txt1_lang, txt1_lang)
    play_text_src_tgt(to_text_txt2, txt2_lang, txt2_lang)
    play_text_src_tgt(to_text_txt3, txt3_lang, txt3_lang)

# Create buttons
start_button = widgets.Button(description="Start Speaking")
start_button.on_click(toggle_recording)

ttt_button = widgets.Button(description="Start TTT")
ttt_button.on_click(ttt_translate)
ttt_button.layout.width = '100px'

tts_button = widgets.Button(description="Start TTS")
tts_button.on_click(tts_translate)
tts_button.layout.width = '100px'

# stop_button = widgets.Button(description="Stop Recording")
# stop_button.on_click(stop_recording)

# Extract the language where 'sym' is equal to txt1_lang
label1 = widgets.Label(value=next((item['language'] for item in lang_list if item['sym'] == txt1_lang), None))
label2 = widgets.Label(value=next((item['language'] for item in lang_list if item['sym'] == txt2_lang), None))
label3 = widgets.Label(value=next((item['language'] for item in lang_list if item['sym'] == txt3_lang), None))
# txt1_lang, lang_list

# Create a text box widget
text_box1 = widgets.Text(value='stt then tts', description='', disabled = True)
text_box2 = widgets.Text(value='stt then tts', description='', disabled = True)
text_box3 = widgets.Text(value='stt then tts', description='', disabled = True)
text_box_ttt = widgets.Text(placeholder='Enter text here to translate to target language from the dropdown...', description='', layout=widgets.Layout(width='500px'))
text_box_tts = widgets.Text(placeholder='Enter text here to translate to target language from the dropdown...', description='', layout=widgets.Layout(width='500px'))


# Display buttons
# Set the layout for the HBox
dropdown_sts.layout.width = '400px'  # Adjust the width as needed
# Set the width for both description label and dropdown box
dropdown_sts.style.description_width = 'initial'  # Use 'initial' to allow the label to expand
hbox_layout = widgets.Layout(justify_content='space-between', width='100%')
hbox = widgets.HBox([start_button, dropdown_sts], layout=hbox_layout)
display(hbox)
# display(widgets.HBox([start_button, dropdown]))
display(widgets.HBox([label1, text_box1, label2, text_box2, label3, text_box3]))

dropdown_t_ttt.layout.width = '200px'  # Adjust the width as needed
# Set the width for both description label and dropdown box
dropdown_t_ttt.style.description_width = 'initial'  # Use 'initial' to allow the label to expand

dropdown_s_ttt.layout.width = '200px'  # Adjust the width as needed
# Set the width for both description label and dropdown box
dropdown_s_ttt.style.description_width = 'initial'  # Use 'initial' to allow the label to expand

hbox_layout = widgets.Layout(justify_content='space-between', width='100%')
hbox_ttt = widgets.HBox([ttt_button,dropdown_s_ttt, text_box_ttt, dropdown_t_ttt], layout=hbox_layout)
display(hbox_ttt)

# tts alone
dropdown_t_tts.layout.width = '200px'  # Adjust the width as needed
# Set the width for both description label and dropdown box
dropdown_t_tts.style.description_width = 'initial'  # Use 'initial' to allow the label to expand

dropdown_s_tts.layout.width = '200px'  # Adjust the width as needed
# Set the width for both description label and dropdown box
dropdown_s_tts.style.description_width = 'initial'  # Use 'initial' to allow the label to expand

hbox_layout = widgets.Layout(justify_content='space-between', width='100%')
hbox_tts = widgets.HBox([tts_button,dropdown_s_tts, text_box_tts, dropdown_t_tts], layout=hbox_layout)
display(hbox_tts)


In [ ]:
SeamlessM4T models support the tasks of:

Speech-to-speech translation (S2ST)
Speech-to-text translation (S2TT)
Text-to-speech translation (T2ST)
Text-to-text translation (T2TT)
Automatic speech recognition (ASR).